# Reshape by ACS table

In [1]:
import numpy as np
import pandas as pd
import intake
import os

In [2]:
df = pd.read_parquet('../data/raw_census_long.parquet')
df = df[df.GEOID=='06037141400']

In [9]:
emp = df[df.table=='emp']

In [10]:
emp

,GEOID,year,variable,value,table,main_var,second_var,new_var,value2
26880,06037141400,2010,S2301_C01_001E,3697.0,emp,pop16,pop16,pop16_pop16,3697.000
26881,06037141400,2010,S2301_C01_010E,2904.0,emp,pop16,white,pop16_white,2904.000
26882,06037141400,2010,S2301_C01_011E,156.0,emp,pop16,black,pop16_black,156.000
26883,06037141400,2010,S2301_C01_012E,0.0,emp,pop16,amerind,pop16_amerind,0.000
26884,06037141400,2010,S2301_C01_013E,372.0,emp,pop16,asian,pop16_asian,372.000
26885,06037141400,2010,S2301_C01_014E,12.0,emp,pop16,pacis,pop16_pacis,12.000
26886,06037141400,2010,S2301_C01_015E,158.0,emp,pop16,other,pop16_other,158.000
26887,06037141400,2010,S2301_C01_016E,95.0,emp,pop16,race2,pop16_race2,95.000
26888,06037141400,2010,S2301_C01_017E,379.0,emp,pop16,hisp,pop16_hisp,379.000
26889,06037141400,2010,S2301_C01_018E,2713.0,emp,pop16,nonhisp,pop16_nonhisp,2713.000


In [7]:
"""
Food stamps table has inconsistent values.
2010-2014, hh in poverty is given in %. same as 
2015-2017, hh in poverty is given as numbers.
Do each table separately.

Store column that tells me if they are numbers, percents, or dollars.
"""
food = df[df.table=='food']
food[food.year==2012]

,GEOID,year,variable,value,table,main_var,second_var,new_var,value2
30174,06037141400,2012,S2201_C01_001E,1969.0,food,hh,hh,hh_hh,1969.0
30175,06037141400,2012,S2201_C01_004E,10.4,food,hh,hh_pov,hh_hh_pov,10.4
30176,06037141400,2012,S2201_C01_016E,105875.0,food,hh,medhhincome,hh_medhhincome,105875.0
30177,06037141400,2012,S2201_C02_001E,13.0,food,hh_food,hh,hh_food_hh,13.0
30178,06037141400,2012,S2201_C02_004E,100.0,food,hh_food,hh_pov,hh_food_hh_pov,100.0
30179,06037141400,2012,S2201_C02_016E,NaN,food,hh_food,medhhincome,hh_food_medhhincome,NaN


In [3]:
def make_percents(row):
    emp_cond = (row.table=='emp') and (row.main_var in ['lf16', 'emp16', 'unempr16'])
    income_cond = (row.table=='income') and (row.main_var=='hh') and (row.second_var != 'hh')
    edu_cond = (row.table=='edu') and (row.second_var != 'pop25') and (row.second_var.find('medearning') == -1)
    food_cond = (row.table=='food') and (row.second_var != 'hh') and (row.second_var.find('income') == -1)
    povfam_cond = (row.table=='povfam') and (row.main_var=='families_pov')
    food_cond = (row.table=='food') and (row.second_var=='hh_pov') and (row.year <= 2014)
    if emp_cond or edu_cond or income_cond or povfam_cond:
        return (row.value/100)
    elif (row.table=='aggpubassist') and (row.value < -100000):
        return np.nan
    else:
        return row.value

df['value2'] = df.apply(make_percents, axis = 1)

In [4]:
df.to_stata('../data/raw_census_long.dta')

In [ ]:
pubassist = df[df.table=='pubassist']
# These are number of families that received food stamps
# Need to derive percents

In [ ]:
pubassist